In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
import random
import gc
from sklearn.model_selection import train_test_split
from sklearn import  metrics
               
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
fy_10 = pd.read_csv('fy_10_augmented.csv')
fy_11 = pd.read_csv('fy_11_augmented.csv')
fy_12 = pd.read_csv('fy_12_augmented.csv')
fy_13 = pd.read_csv('fy_13_augmented.csv')
fy_14 = pd.read_csv('fy_14_augmented.csv')
fy_15 = pd.read_csv('fy_15_augmented.csv')
fy_16 = pd.read_csv('fy_16_augmented.csv')

fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_15, fy_16]

del fy_10
del fy_11
del fy_12
del fy_13
del fy_14
del fy_15
del fy_16
gc.collect()

essentials = ['Unnamed: 0',
 'Fund Id',
 'Business Area',
 'Fund Center Id',
 'GL Account',
 'GL Description',
 'GL Category',
 'GL Category Description',
 'Revenue or Expenditure',
 'Fiscal Year',
 'Fund Name',
 'Fund Type Descr',
 'Business Area Name',
 'Fund Center Name',
 'Commitment Set Description',
 'Original Budget',
 'Current Budget',
 'Actuals',
 '3Y_Past_Actuals',
 '2Y_Past_Actuals',
 '1Y_Past_Actuals',
 '2Y_1Y_Actuals_Mean',
 '3Y_2Y_1Y_Actuals_Mean',
 '3Y_Past_Original_Budget',
 '2Y_Past_Original_Budget',
 '1Y_Past_Original_Budget',
 '2Y_1Y_Original_Budget_Mean',
 '3Y_2Y_1Y_Original_Budget_Mean',
 '3Y_Past_OA_Difference',
 '2Y_Past_OA_Difference',
 '1Y_Past_OA_Difference',
 '2Y_1Y_OA_Difference_Mean',
 '3Y_2Y_1Y_OA_Difference_Mean',
 'OA_Difference',
 'Yearly Crime Index',
 'Yearly Avg High Temp',
 'Yearly Avg Low Temp',
 'Yearly Rain Fall',
 'Astros Playoff Rounds',
 'Astros Yearly Attendance',
 'Rockets Playoff Rounds',
 'Texans Playoff Rounds',
 'Texans Average Attendance',
 'Rockets Total Attendance']

dummies = ['Fund Name_Asset Forfeiture Fund - Justice',
 'Fund Name_Asset Forfeiture Fund - State',
 'Fund Name_Auto Dealers Fund',
 'Fund Name_Building Inspection Fund',
 'Fund Name_C&E-Civic Center Facility Revenue Fund',
 'Fund Name_Cable Television Fund',
 'Fund Name_Child Safety Fund',
 'Fund Name_Digital Houston Special Revenue',
 'Fund Name_Expansion of Recycling Program',
 'Fund Name_Fleet Management Fund',
 'Fund Name_GSD - In-House Renovation Revolving Fund',
 'Fund Name_General Fund',
 'Fund Name_Greater Houston Trans & Emer Center',
 'Fund Name_HAS-Revenue Fund',
 'Fund Name_Health Benefits Fund',
 'Fund Name_Historic Preservation Fund',
 'Fund Name_Houston Emergeny Center',
 'Fund Name_Long-Term Disablility Plan Fund',
 'Fund Name_Municipal Court Technology Fund',
 'Fund Name_PWE-Combined Utility System Gen Pur Fund',
 'Fund Name_PWE-Combined Utility System Oper Fund',
 'Fund Name_PWE-W & S System Operating Fund',
 'Fund Name_Park Special Revenue Fund',
 'Fund Name_Parking Mangement Operating Fund',
 'Fund Name_Police Special Services',
 'Fund Name_Project Cost Recovery',
 'Fund Name_Property & Casualty Fund',
 'Fund Name_Stormwater Fund',
 'Fund Name_Supplemental Environmental Proj',
 'Fund Name_Workers Compensation Admin Fund',
 'Fund Type Descr_Enterprise Funds',
 'Fund Type Descr_General Funds',
 'Fund Type Descr_Internal Services Funds',
 'Fund Type Descr_Special Revenue Funds',
 'Business Area Name_Admn. & Regulatory Affairs',
 'Business Area Name_Business Opportunity',
 "Business Area Name_City Controller's Office",
 'Business Area Name_City Council',
 'Business Area Name_City Secretary',
 'Business Area Name_Citywide General Government',
 'Business Area Name_Convention and Entertainment',
 'Business Area Name_Finance',
 'Business Area Name_General Debt Services',
 'Business Area Name_General Services Department',
 'Business Area Name_Health & Human Services',
 'Business Area Name_Housing and Community Devp.',
 'Business Area Name_Houston Airport System (HAS)',
 'Business Area Name_Houston Emergency Center (HEC)',
 'Business Area Name_Houston Fire Department (HFD)',
 'Business Area Name_Houston Police Department',
 'Business Area Name_Human Resources Dept.',
 'Business Area Name_Information Technology (IT)',
 'Business Area Name_Legal Department',
 'Business Area Name_Library',
 "Business Area Name_Mayor's Office",
 'Business Area Name_Municipal Court Judicial',
 'Business Area Name_Municipal Courts Department',
 'Business Area Name_Parks & Recreation',
 'Business Area Name_Planning & Development',
 'Business Area Name_Public Works & Engineering-PWE',
 'Business Area Name_Solid Waste Management',
 'Commitment Set Description_50 - Total Expenditures',
 'Business Area Name_Houston Police Department-HPD',
 'Business Area Name_Houston Information Tech Svcs'
]

drop_list = [
    
    'Fund Name', 
    'Fund Type Descr', 
    'Business Area Name',
    'Commitment Set Description',
    'GL Description',
    'GL Category Description',
    'Revenue or Expenditure',
    'Fund Type Descr',
    'Fund Center Name'
]


for i, y in enumerate(fy_list):
    fy_list[i] = pd.concat([y[essentials], y[dummies]], axis=1)
    fy_list[i]['Over_Budget'] = fy_list[i]['OA_Difference']>0
    
for i, y in enumerate(fy_list):
    if i==0:
        fy_list[i]["1YP_OA_Diff_Over_80K"]=999999999999999
    else:
        fy_list[i]["1YP_OA_Diff_Over_80K"]=0
        over80k_index = fy_list[i].loc[(fy_list[i]['1Y_Past_OA_Difference'].apply(lambda x: abs(x))>80000.0)].index
        fy_list[i].loc[over80k_index, '1YP_OA_Diff_Over_80K'] = 1
        

        
        
splits_to_try = [
                 [(0,3),(3,4),'10-12 to predict 13'],
                 [(0,3),(3,5),'10-12 to predict 13 and 14'],
                 [(1,4),(4,5),'11-13 to predict 14'], 
                 [(1,4),(4,6),'11-13 to predict 14 and 15'],
                 [(2,5),(5,6),'12-14 to predict 15'],
                 [(2,5),(5,7),'12-14 to predict 15 and 16'],
                 [(3,6),(6,7),'13-15 to predict 16'], 
                 [(0,4),(5,6),'10-13 to predict 14'],
                 [(0,4),(4,6),'10-13 to predict 14 and 15'],
                 [(0,4),(4,7),'10-13 to predict 14 and 15 and 16'],
                 [(1,5),(5,6),'11-14 to predict 15'],
                 [(1,5),(5,7),'11-14 to predict 15 and 16'],
                 [(2,6),(6,7),'12-15 to predict 16']
                ]
        
        

In [ ]:
hold_list = []
for split in splits_to_try:
    
    hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Actuals']
    X_train.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Actuals']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBRFClassifier(subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)

    model_test_auc = metrics.roc(y_test, xgb_test_predictions)
    model_train_auc = metrics.roc(y_train, xgb_train_predictions)
    original_budget_auc = metrics.roc(y_test, y_original_budget)
    
    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_auc': model_test_auc})
    hold_dict.update({'model_train_auc': model_train_auc})
    hold_dict.update({'original_budget_auc': original_budget_auc})
    
    
    hold_list.append(hold_dict)
    
   




In [ ]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['model_test_auc']-results_df['original_budget_auc']
results_df


In [ ]:
results_df.loc[[0,2,4,6,7,10]]['Model Improvement']

In [ ]:
hold_list = []
for split in splits_to_try:
    
    hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Actuals']
    X_train.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Actuals']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBClassifier(subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)

    model_test_auc = metrics.roc(y_test, xgb_test_predictions)
    model_train_auc = metrics.roc(y_train, xgb_train_predictions)
    original_budget_auc = metrics.roc(y_test, y_original_budget)
    
    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_auc': model_test_auc})
    hold_dict.update({'model_train_auc': model_train_auc})
    hold_dict.update({'original_budget_auc': original_budget_auc})
    
    
    hold_list.append(hold_dict)
    
   




In [ ]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['model_test_auc']-results_df['original_budget_auc']
results_df


In [ ]:
results_df.loc[[0,2,4,6,7,10]]['Model Improvement']


In [129]:
results_df = pd.DataFrame(hold_list)
results_df

,model_test_RMSE,model_train_RMSE,original_budget_RMSE
0,439127.123743,251443.840946,573679.154344
1,548996.930085,293378.015533,808664.924708
2,391419.495231,289915.446537,549885.577426


In [101]:
hold_dict.keys()

dict_keys(['model', 'xgb_test_predictions', 'xgb_train_predictions', 'model_RMSE', 'original_budget_RMSE'])

In [86]:
len(res)

502290.7952341024

In [87]:
model_sum_error = np.sum(abs(xgb_test_predictions-y_test))
model_sum_error

1337609774.0442579

In [88]:
budget_sum_error = np.sum(abs(y_original_budget-y_test))
budget_sum_error

1154587673.21

### Model precision over budget

In [89]:
print('$',round(budget_sum_error-model_sum_error,2))

$ -183022100.83


592194.454153469

In [91]:
y_test.std()

6244039.420088603

In [ ]:
sns.set_style('whitegrid')

x = np.linspace(-1000000000,1000000000,1000000000)
plt.figure(figsize=(12,12))
plt.scatter(y_train, xgb_train_predictions, c='red', alpha=.95)
plt.scatter(y_test, xgb_test_predictions, c='blue', alpha=.95)
plt.scatter(y_test, y_original_budget, c='green', alpha=.95)
plt.plot(x,x,c='black')
plt.xlim([-5,1000000000])
plt.ylim([-5,1000000000])
plt.xlabel('observations')
plt.ylabel('predictions')


In [130]:
weight_dict = model.get_booster().get_score(importance_type="weight")
weight_df = pd.DataFrame(weight_dict, index=range(len(weight_dict)))
weight_df.T[0].sort_values(ascending=False)

Original Budget                                    263
1Y_Past_Actuals                                     56
1Y_Past_OA_Difference                               41
Fiscal Year                                         32
3Y_Past_OA_Difference                               28
2Y_Past_Actuals                                     28
Unnamed: 0                                          28
GL Account                                          12
3Y_Past_Actuals                                     11
3Y_2Y_1Y_OA_Difference_Mean                          8
2Y_Past_OA_Difference                                7
Fund Name_PWE-Combined Utility System Oper Fund      5
2Y_1Y_Original_Budget_Mean                           5
2Y_1Y_OA_Difference_Mean                             4
1Y_Past_Original_Budget                              4
Fund Name_Property & Casualty Fund                   4
3Y_2Y_1Y_Actuals_Mean                                3
Rockets Total Attendance                             1
Yearly Avg

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test , xgb_test_predictions)

In [ ]:

sns.set_style('darkgrid')
sns.set(font_scale=2.5)
bins = np.arange(0,90000,90000/50)
 
fig, ax = plt.subplots(figsize=(20,15))

sns.distplot(y_test, label='2015 Actuals',color='green', norm_hist=True, bins=bins, kde=False,  ax = ax)
sns.distplot(xgb_test_predictions, label='XGB_test_predictions 2015',color='black', norm_hist=True, bins=bins, kde=True,  ax = ax)
sns.distplot(y_original_budget, label='2015_original_budget',color='blue',norm_hist=True, bins=bins, kde=True,  ax = ax)
    
ax.set_title(" ")
ax.set_ylabel(" ")
ax.set_xlabel(" ")
ax.set_xlim(0,90000)
ax.legend()

In [ ]:
original_budget_residuals = fy_15['Original Budget']-fy_15['Actuals']
model_residuals = xgb_test_predictions-y_test
hold_df = pd.DataFrame()
hold_df['Original Budget Residuals'] = original_budget_residuals
hold_df['Model Residuals'] = model_residuals


In [ ]:
np.sqrt(np.mean(model_residuals**2))

In [ ]:
np.sqrt(np.mean(original_budget_residuals**2))

In [ ]:
model_residuals_series = pd.Series(model_residuals)

In [ ]:
original_budget_residuals_series =pd.Series(original_budget_residuals)

In [ ]:
residuals_df = pd.DataFrame()
residuals_df['Model_Residuals'] = model_residuals_series
residuals_df['Original_Budget_Residuals'] = original_budget_residuals_series

extreme_residuals = residuals_df.loc[(residuals_df['Model_Residuals']>=80000) | (residuals_df['Model_Residuals']<=-80000) | (residuals_df['Original_Budget_Residuals']>=80000) | (residuals_df['Original_Budget_Residuals']<=-80000) ] 
residuals_under_80K = residuals_df.loc[(residuals_df['Model_Residuals']<80000) & (residuals_df['Model_Residuals']>-80000) & (residuals_df['Original_Budget_Residuals']<=80000) & (residuals_df['Original_Budget_Residuals']>=-80000) ] 


In [ ]:
print('Model Extreme Resisduals:           ',int(np.sqrt(np.mean(extreme_residuals['Model_Residuals']**2))), '\nOriginal Budget Extreme Resisduals: ',  int(np.sqrt(np.mean(extreme_residuals['Original_Budget_Residuals']**2))))
print(' ')
print('Model Under 80K Resisduals:           ',int(np.sqrt(np.mean(residuals_under_80K['Model_Residuals']**2))), '\nOriginal Budget Under 80K Resisduals: ',  int(np.sqrt(np.mean(residuals_under_80K['Original_Budget_Residuals']**2))))


In [ ]:
sns.set_style('darkgrid')
sns.set(font_scale=2.5)
 
fig, ax = plt.subplots(figsize=(20,15))
bins_ = np.arange(-80000,80000,80000/50)

sns.distplot(model_residuals, label='Model Residuals', norm_hist=True, bins = bins_, color='green',   ax = ax)
sns.distplot(original_budget_residuals, label='Original Budget Residuals',bins=bins_, color='black',  ax = ax)
    
ax.set_title(" ")
ax.set_ylabel(" ")
ax.set_xlabel(" ")
ax.set_xlim(-80000,80000)
ax.legend()

In [131]:
#reference: https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [150]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


train_df = pd.concat([fy_list[0], fy_list[1], fy_list[2], fy_list[3], fy_list[4]])
Y = train_df['Actuals'].values
X = train_df.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1)
test_df = pd.concat([fy_list[5], fy_list[6]])
test = test_df.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1)
y_test = test_df['Actuals']
original_budget = test_df['Original Budget']

In [136]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [137]:
model = xgb.XGBRegressor(learning_rate=.02, n_estimators=600, silent=True, nthread=1)

#xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
  #                  silent=True, nthread=1)

In [139]:
folds = 3
param_comb = 5

#skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, n_jobs=4, verbose=3, random_state=1001 )

#cv=skf.split(X,Y)
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, Y)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:  8.2min finished



 Time taken: 0 hours 10 minutes and 51.47 seconds.


In [140]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([ 91.14354022, 139.68440715, 137.26617638, 110.86528158,
       119.09287612]), 'std_fit_time': array([2.81644458, 5.72055445, 1.28532953, 2.43847861, 4.22122272]), 'mean_score_time': array([0.33134842, 0.52923695, 0.52825665, 0.53919148, 0.35074592]), 'std_score_time': array([0.04293319, 0.05562085, 0.07539545, 0.14604426, 0.02436338]), 'param_subsample': masked_array(data=[1.0, 0.6, 0.8, 1.0, 0.8],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 1, 5, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 5, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[5, 1.5, 1, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?

In [146]:
xgb_test_predictions = random_search.predict(test)
#results_df = pd.DataFrame(data={'Actuals':y_test[:,1]})
#results_df.to_csv('submission-random-grid-search-xgb-porto-01.csv', index=False)


In [199]:
model_test_RMSE = np.sqrt(np.mean((xgb_test_predictions-y_test)**2))
#model_train_RMSE = np.sqrt(np.mean((xgb_train_predictions-y_train)**2))
original_budget_RMSE = np.sqrt(np.mean((original_budget-y_test)**2))


In [200]:
y_test.shape


(22998,)

In [201]:
len(y_original_budget)


26563

In [196]:
a =pd.Series([7,8,9,10])
b=pd.Series([1,2,1])
np.mean([36,36,64])
np.mean((a-b)**2)
original_budget[:10]


0     239999
1      50000
2        140
3     107802
4     418375
5       5076
6       4000
7     107875
8       3500
9    1628293
Name: Original Budget, dtype: int64

In [197]:
y_test[:10]

0     217011.80
1      38227.23
2        129.30
3      99487.79
4     345056.07
5       4494.92
6       2594.24
7      87066.07
8          0.00
9    1458631.47
Name: Actuals, dtype: float64

In [167]:
pd.DataFramey_original_budget[:1000].values



array([ 6.000000e+03,  1.532743e+06,  9.144000e+03,  2.102400e+04,
        1.440000e+02,  0.000000e+00,  0.000000e+00,  4.443000e+03,
        1.250000e+03,  0.000000e+00,  7.000000e+04,  1.000000e+04,
        3.230000e+03,  0.000000e+00,  1.009000e+03,  6.692000e+03,
        8.708280e+05,  1.060000e+04,  1.886000e+05,  4.630000e+02,
        1.000000e+03,  4.246900e+04,  2.393000e+04,  8.500000e+02,
        2.457460e+05,  4.800000e+04,  0.000000e+00,  1.000000e+03,
        3.000000e+03,  0.000000e+00,  3.158400e+04,  0.000000e+00,
        2.800000e+01,  7.500000e+03,  1.137480e+05,  0.000000e+00,
        1.133100e+05,  7.384000e+04,  5.037000e+03,  2.500000e+04,
        4.121600e+04,  1.357000e+03,  1.000000e+02,  8.500000e+02,
        4.500000e+04,  0.000000e+00,  1.000000e+03,  2.000000e+04,
        1.569100e+04,  1.643000e+03,  2.000000e+03,  1.600000e+02,
        1.164000e+04,  9.100000e+03,  2.550000e+02,  2.791690e+05,
        1.700000e+02,  1.000000e+02,  2.400000e+03,  0.000000e

In [202]:
y_test.std()


6257843.454195455

In [203]:
model_test_RMSE


684182.9834646059

In [204]:
original_budget_RMSE


519179.9135619907

In [158]:
weight_dict = random_search.best_estimator_.get_booster().get_score(importance_type="weight")
weight_df = pd.DataFrame(weight_dict, index=range(len(weight_dict)))
weight_df.T[0].sort_values(ascending=False)


Original Budget                                       1988
1Y_Past_Actuals                                        542
1Y_Past_OA_Difference                                  432
Unnamed: 0                                             382
GL Account                                             214
Fiscal Year                                            210
3Y_Past_Actuals                                        150
2Y_Past_OA_Difference                                  140
2Y_Past_Actuals                                        113
Fund Id                                                105
2Y_1Y_OA_Difference_Mean                               102
1Y_Past_Original_Budget                                 68
3Y_Past_OA_Difference                                   59
3Y_2Y_1Y_OA_Difference_Mean                             57
2Y_Past_Original_Budget                                 49
Business Area                                           49
3Y_2Y_1Y_Actuals_Mean                                   

In [160]:
len(y_original_budget)

26563

In [159]:
len(xgb_test_predictions)

22998

In [161]:
len(y_test)

22998

In [ ]:
sns.set_style('whitegrid')

x = np.linspace(-1000000000,1000000000,1000000000)
plt.figure(figsize=(12,12))
#plt.scatter(y_train, xgb_train_predictions, c='red', alpha=.95)
plt.scatter(y_test, xgb_test_predictions, c='blue', alpha=.95)
plt.scatter(y_test, y_original_budget, c='green', alpha=.95)
plt.plot(x,x,c='black')
plt.xlim([-5,1000000000])
plt.ylim([-5,1000000000])
plt.xlabel('observations')
plt.ylabel('predictions')


# H2o

In [12]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: Picked up _JAVA_OPTIONS: -Xverify:none; java version "12.0.1" 2019-04-16; Java(TM) SE Runtime Environment (build 12.0.1+12); Java HotSpot(TM) 64-Bit Server VM (build 12.0.1+12, mixed mode, sharing)
  Starting server from /anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/0y/vcjmh6651517f16c4qtnxyg00000gq/T/tmppisi9qtu
  JVM stdout: /var/folders/0y/vcjmh6651517f16c4qtnxyg00000gq/T/tmppisi9qtu/h2o_josephdown_started_from_python.out
  JVM stderr: /var/folders/0y/vcjmh6651517f16c4qtnxyg00000gq/T/tmppisi9qtu/h2o_josephdown_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,1 month and 10 days
H2O cluster name:,H2O_from_python_josephdown_97mll2
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [13]:
# create h2o frame for training data
training_frame = pd.concat([X_train,y_train], axis=1)
training_frame.columns = list(training_frame.columns[:-1]) + ['Actuals']
training_frame = h2o.H2OFrame(training_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
# create h2o frame for testing data
testing_frame = pd.concat([X_test,y_test], axis=1)
testing_frame.columns = list(testing_frame.columns[:-1]) + ['Actuals']
testing_frame = h2o.H2OFrame(testing_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(None, 'Actuals', training_frame)

AutoML progress: |██████████████████████████████████████

In [ ]:
y_test.std()

In [ ]:
predicted_actuals = aml.predict(h2o.H2OFrame(pd.concat([X_test, y_test], axis=1)))
np.sqrt(np.mean((predicted_actuals['predict'].as_data_frame()['predict'].values-y_test)**2))

In [ ]:
aml.leader

In [ ]:
#3:55 pm

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_train, aml.predict(training_frame).as_data_frame()['predict'].values, c='red')
plt.scatter(y_test, aml.predict(testing_frame).as_data_frame()['predict'].values, c='blue')
#plt.scatter(train_df['voyage_duration'], train_df['eta_diff'], c='green')
plt.plot(x,x,c='black')
plt.xlim([-5,100000000])
plt.ylim([-5,100000000])
plt.xlabel('observations')
plt.ylabel('predictions')

In [ ]:
#9/4 4:42pm